In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pickle
from sklearn.externals import joblib

In [3]:
batch_size = 256

In [4]:
x = tf.placeholder(
    shape=[batch_size, 50, 64],
    dtype=tf.float32
)
y = tf.placeholder(
    shape=[batch_size, 1],
    dtype=tf.float32,
)

In [5]:
hidden_size = 64
fw_lstm_cell_1 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)
fw_lstm_cell_2 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)
bw_lstm_cell_1 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)
bw_lstm_cell_2 = tf.contrib.rnn.LSTMCell(
    num_units=hidden_size, 
)
outputs, final_fw, final_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
    [fw_lstm_cell_1],#,fw_lstm_cell_2],
    [bw_lstm_cell_1],#,bw_lstm_cell_2],
    x,
    dtype=tf.float32
)

In [6]:
final_fw =  tf.concat( [final_fw[0][0],final_fw[0][1]],axis=1)
final_bw =  tf.concat( [final_bw[0][0],final_bw[0][1]],axis=1)
final = tf.concat( [final_fw, final_bw], axis=1)

In [7]:
hidden = tf.layers.Dense(units=256, activation=tf.nn.elu)(final)

In [8]:
output = tf.layers.Dense(units=1, activation=None)(hidden)
# output = tf.layers.Dense(units=1, activation=None)(last_states)
# output = tf.layers.Dense(units=4, activation=None)(final)

In [9]:
mae = tf.reduce_mean(tf.abs(output-y))
mse = tf.reduce_mean(tf.pow(output-y, 2))

rmae = tf.sqrt(tf.reduce_mean(tf.abs(output-y)))
rmse = tf.sqrt(tf.reduce_mean(tf.pow(output-y, 2)))


In [10]:
optimizer = tf.train.AdamOptimizer(
    learning_rate=0.0001
)
train_op = optimizer.minimize(mse)

In [11]:
# Слова в индексы эмбеддинга

In [12]:
import gensim
word2vec = gensim.models.Word2Vec.load('./../topic7-word2vec/word2vec32')

In [13]:
# path = './../topic7-word2vec/normalized_train.csv' 

# with open('actual_train.csv','w') as f:
#     with open(path, 'r') as f_norm_text:
#         with open('./../dz4/big_train.csv', 'r') as f_rating:
#             print(f_rating.readline())
#             for norm_text_line, rating_line in zip(f_norm_text, f_rating):
#                 data = rating_line.strip().split(',')
#                 _id,context_id,label,text,likes,rating = data

#                 if rating == 'n/a':
#                     continue
                    
#                 f.write('%s,%s\n' % (norm_text_line.split(',')[0].strip(), rating))
                
# # path = './../topic7-word2vec/normalized_test.csv'

In [14]:
word_dictionary = {w:i for (i,w) in enumerate(word2vec.wv.index2word)}

In [15]:
!wc actual_train.csv

   84070 5214415 73895248 actual_train.csv


In [16]:
def iter_data(batch_size=batch_size, is_train=True, is_val=True):
    max_length = 50
    batch_x, batch_y = [], []
    if is_train:
        path = 'actual_train.csv' 
    else:
        path = 'actual_test.csv'
        
    with open(path, 'r') as f:
        for _idx, line in enumerate(f):
            
            if is_train and is_val and _idx > 4200:
                break
                
            if is_train and (not is_val) and _idx < 4200: #> 84070-4200:
                continue
                
            if is_train:
                text, label = line.strip().split(',')
            else:
                text, label = line.strip().split(',')
            indexes = [
                word_dictionary.get(word) 
                for word in text.split() 
                if word in word_dictionary
            ]
            if not indexes:
                word_vectors = np.zeros( (max_length, 64), dtype=np.float32)
            else:
                word_vectors = word2vec.wv.syn0[indexes]

            if word_vectors.shape[0]< max_length:
                extra = np.array([
                    [0 for i in range(64)]
                    for j in range(max_length-word_vectors.shape[0])
                ])
                word_vectors = np.vstack(
                    [
                        word_vectors, extra
                    ]
                )

            if word_vectors.shape[0]> max_length:
                word_vectors = word_vectors[:max_length]
                
            batch_x.append(
                word_vectors
            )
            
            if is_train:
#                 _label_vec = np.zeros(4)
#                 _label_vec[int(label)] = 1
                batch_y.append([int(label)])
            
            if len(batch_x) >=batch_size:
                yield np.array(batch_x), np.array(batch_y)
                batch_y, batch_x = [],[]
                
    if is_train==False:

        while len(batch_x)<batch_size:
            batch_x.append(
                np.zeros( (max_length, 64), dtype=np.float32)
            )
            
        yield np.array(batch_x), np.array(batch_y)

In [17]:
def eval_target(sess, filename):
    Y_= []
    for _tick, (_x,_y) in enumerate(iter_data(is_train=False)):

        _output = sess.run(
            [output],
            feed_dict={
                x: _x,
            }
        )
        for probs in _output[0]:
            Y_.append(probs)
    Y_= np.array(Y_)
    with open('%s.csv' % filename,'w') as sol:
        with open('../dz4/rating_test_without_rating.csv', 'r') as f:
            sol.write('_id,rating\n')
            f.readline()
            for (line, label) in zip(f, Y_):
                data = line.split(',')
                sol.write('%s,%s\n' % (data[0], label[0]))

In [34]:
min_mse = 2
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, save_path='./rating_rnn.model')
    try:
        for epoch in range(20,30):
            for _tick, (_x,_y) in enumerate(iter_data(is_train=True,is_val=False)):

                _, _mae, _mse, _rmae, _rmse = sess.run(
                    [train_op, mae, mse, rmae, rmse ],
                    feed_dict={
                        x: _x,
                        y: _y
                    }
                )

                if _tick and not _tick % 100:
                    
                    loss_ = []
                    for (_x,_y) in iter_data(is_train=True,is_val=True):
                        loss_list = sess.run(
                            [mae, mse, rmae, rmse],
                            feed_dict={
                                x: _x,
                                y: _y,
                            }
                        )
                        loss_.append(loss_list)
                    loss_ = np.array(loss_)
                    loss_ = loss_.mean(axis=0)
                    print(epoch,_tick)
                    print(
                        '%3.2f(%3.2f), %3.2f(%3.2f), %3.2f(%3.2f), %3.2f(%3.2f)' % (
                            _mae,loss_[0],
                            _mse,loss_[1],
                            _rmae,loss_[2],
                            _rmse,loss_[3]
                        )
                    )
                    
                    if min_mse>loss_[1]:
                        if epoch< 2: continue
                        min_mse = loss_[1]
                        eval_target(sess, 'rating_2xbi-lstm_%s.csv' % min_mse)
                            
                    saver.save(sess=sess, save_path='./rating_rnn.model')
                    
                    
                    
            saver.save(sess=sess, save_path='./rating_rnn.model')

    except KeyboardInterrupt:
        saver.save(sess=sess, save_path='./rating_rnn.model')
        raise KeyboardInterrupt

INFO:tensorflow:Restoring parameters from ./rating_rnn.model
20 100
0.55(0.62), 0.52(0.74), 0.74(0.78), 0.72(0.86)
20 200
0.48(0.59), 0.48(0.73), 0.69(0.77), 0.70(0.85)
20 300
0.47(0.59), 0.44(0.73), 0.68(0.77), 0.67(0.86)
21 100
0.54(0.62), 0.52(0.74), 0.74(0.78), 0.72(0.86)
21 200
0.47(0.59), 0.48(0.73), 0.69(0.77), 0.69(0.85)
21 300
0.46(0.59), 0.44(0.73), 0.68(0.77), 0.66(0.86)
22 100
0.54(0.62), 0.51(0.74), 0.73(0.78), 0.71(0.86)
22 200
0.47(0.59), 0.47(0.73), 0.68(0.76), 0.69(0.85)
22 300
0.46(0.59), 0.43(0.74), 0.68(0.77), 0.66(0.86)
23 100
0.54(0.62), 0.50(0.74), 0.73(0.79), 0.71(0.86)
23 200
0.46(0.59), 0.46(0.73), 0.68(0.76), 0.68(0.85)
23 300
0.45(0.59), 0.42(0.74), 0.67(0.77), 0.65(0.86)
24 100
0.53(0.62), 0.49(0.74), 0.73(0.79), 0.70(0.86)
24 200
0.46(0.59), 0.46(0.73), 0.68(0.76), 0.68(0.85)
24 300
0.45(0.59), 0.42(0.74), 0.67(0.77), 0.64(0.86)
25 100
0.53(0.62), 0.49(0.74), 0.73(0.79), 0.70(0.86)
25 200
0.45(0.58), 0.45(0.73), 0.67(0.76), 0.67(0.86)
25 300
0.44(0.59), 0.

In [31]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_path='./rating_rnn.model')
    
    eval_target(sess, 'rating_rnn_')

INFO:tensorflow:Restoring parameters from ./rating_rnn.model


In [43]:
Y_= np.array(Y_)

In [44]:
Y_.shape

(83328, 1)

In [45]:

with open('rating_rnn_.csv','w') as sol:
    with open('../dz4/rating_test_without_rating.csv', 'r') as f:
        sol.write('_id,rating\n')
        f.readline()
        for (line, label) in zip(f, Y_):
            data = line.split(',')
            sol.write('%s,%s\n' % (data[0], label[0]))
#             yield preprocessing(line.strip().split(',')[1].lower()).split()

In [19]:
!open .

In [60]:
!wc ../dz4/rating_test_without_rating.csv

   83221 7245744 84404527 ../dz4/rating_test_without_rating.csv


In [27]:
import os
D = []
for fname in os.listdir('.'):
    Y = []
    if fname[:6]=='rating' and fname[-3:]=='csv':
        with open(fname, 'r') as f:
            print(f.readline())
            for line in f:
                _id, val = line.split(',')
                Y.append(float(val))
        D.append(Y)
    else:
        continue

D = np.array(D)

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating

_id,rating



In [31]:
# for 

In [33]:
with open('concat_rating.csv','w') as sol:
    with open('../dz4/rating_test_without_rating.csv', 'r') as f:
        sol.write('_id,rating\n')
        f.readline()
        for (line, label) in zip(f, D.mean(axis=0)):
            data = line.split(',')
            sol.write('%s,%s\n' % (data[0], label))